# Llama 3.2 1B Medical Chatbot Fine-tuning
Fixed version with correct dataset columns and training configuration

In [1]:
# List input files
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/llama-3.2/transformers/1b-instruct/1/config.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/README.md
/kaggle/input/llama-3.2/transformers/1b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.2/transformers/1b-instruct/1/tokenizer.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/LICENSE.txt
/kaggle/input/llama-3.2/transformers/1b-instruct/1/model.safetensors
/kaggle/input/llama-3.2/transformers/1b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/.gitattributes
/kaggle/input/llama-3.2/transformers/1b-instruct/1/generation_config.json


In [2]:
# Install required packages with specific versions
%pip install -q \
  transformers==4.46.2 \
  datasets==3.1.0 \
  accelerate==1.1.1 \
  peft==0.13.2 \
  trl==0.12.0 \
  bitsandbytes==0.44.1 \
  wandb==0.18.6 \
  safetensors==0.4.5 \
  huggingface-hub==0.25.1 \
  tokenizers==0.20.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import required libraries
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import torch
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Configuration
base_model = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model_name = "llama-3-1b-medical-chatbot-v1"

In [5]:
# QLoRA Configuration
qlora_configs = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [6]:
# Load Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=qlora_configs,
    device_map="auto",
)

In [7]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [8]:
# LoRA Configuration
lora_configs = LoraConfig(
    r=8,  # Increased from 4 for better performance
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],  # Targeting attention layers
)

In [9]:
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_configs)
model.print_trainable_parameters()

trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


In [10]:
# Load and explore dataset
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.shuffle(seed=42).select(range(1000))  # Using 1000 samples

print("Dataset columns:", dataset.column_names)
print("\nSample data:")
print(dataset[0])

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

Dataset columns: ['Description', 'Patient', 'Doctor']

Sample data:
{'Description': 'Appendix cancer, adino carcinoma, removed partial intestine. Suggestion?', 'Patient': 'last year my wife was went through a surgery for appendix cancer, that appendix was removed , that appendix slice tested in lab and found so called adino carcinoma in apedix,  after that doctor decided to operate again and remove her partial intestine, there was no sign of cancer in any test other than the biopsy of appendix, however  after one moth of hospitalization came back to home, 6 month of follow up check up no bad sign, now almost one year of surgery puss mark notice at steches near belly button .Please advice this is not a sign of any cancer', 'Doctor': 'Hi and welcome to HCM. First, you dont have to worry. This cant be tumour relaps because this is lesion in abdominall wall,obviously some local infection or wound abscess.This is often seen after laparotomy. Appendix cancers are rare but in most cases surge

In [11]:
# Format dataset with correct column names
def format_chat_template(row):
    # Combine Description and Patient message as user input
    user_content = f"{row['Description']}\n\nPatient: {row['Patient']}"
    
    chat_format = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": row["Doctor"]}
    ]
    
    # Apply chat template
    row["text"] = tokenizer.apply_chat_template(
        chat_format, 
        tokenize=False,
        add_generation_prompt=False
    )
    
    # Tokenize
    encoding = tokenizer(
        row["text"], 
        padding='max_length', 
        truncation=True, 
        max_length=512,
        return_tensors=None
    )
    
    row["input_ids"] = encoding["input_ids"]
    row["attention_mask"] = encoding["attention_mask"]
    row["labels"] = encoding["input_ids"].copy()
    
    return row

# Apply formatting
print("Formatting dataset...")
dataset = dataset.map(
    format_chat_template,
    num_proc=4,  # Reduced from 16 for stability
    remove_columns=dataset.column_names,  # Remove original columns
)

print("\nFormatted dataset columns:", dataset.column_names)
print("Sample formatted text:")
print(dataset[0]['text'][:500] + "...")

Formatting dataset...


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]


Formatted dataset columns: ['text', 'input_ids', 'attention_mask', 'labels']
Sample formatted text:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Appendix cancer, adino carcinoma, removed partial intestine. Suggestion?

Patient: last year my wife was went through a surgery for appendix cancer, that appendix was removed , that appendix slice tested in lab and found so called adino carcinoma in apedix,  after that doctor decided to operate again and remove her partial intestine, there was no sign of cancer in any test other than the biopsy of appendix, however  after one moth of hos...


In [12]:
# Split dataset
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"Train size: {len(dataset['train'])}")
print(f"Test size: {len(dataset['test'])}")

Train size: 900
Test size: 100


In [13]:
# Training Arguments
training_arguments = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",  # Fixed: using eval_strategy instead of evaluation_strategy
    eval_steps=50,  # Changed from 0.2 to actual steps
    logging_steps=10,
    save_steps=50,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True,
    remove_unused_columns=False,  # CRITICAL: Must be False
    report_to="none",  # Disable wandb
)

In [14]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

In [15]:
# Disable WANDB
os.environ["WANDB_DISABLED"] = "true"

In [16]:
# Start training
print("Starting training...")
trainer.train()

Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
50,1.430500,1.490436
100,1.462500,1.439240
150,1.497300,1.424049
200,1.439200,1.417485


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=225, training_loss=1.688678690592448, metrics={'train_runtime': 828.9256, 'train_samples_per_second': 1.086, 'train_steps_per_second': 0.271, 'total_flos': 2692909891584000.0, 'train_loss': 1.688678690592448, 'epoch': 1.0})

In [17]:
# Save the fine-tuned model
print("Saving model...")
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
print(f"Model saved to {new_model_name}")

Saving model...
Model saved to llama-3-1b-medical-chatbot-v1


In [18]:
# Test the model
print("Testing the fine-tuned model...\n")

test_messages = [
    {
        "role": "user",
        "content": "I have been experiencing severe headaches and dizziness for the past week. What could be causing this?"
    },
]

# Prepare input
prompt = tokenizer.apply_chat_template(
    test_messages, 
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(model.device)

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract just the assistant's response
if "assistant" in response:
    response = response.split("assistant")[-1].strip()

print("Response:")
print(response)

Testing the fine-tuned model...

Response:
Hello, welcome to Health Care Magic.Hope you are feeling fine and not experiencing any severe symptoms like dizziness and headaches. If you are experiencing dizziness and severe headaches, please consult your doctor.
